In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from datetime import datetime
PATH = ChromeDriverManager().install()
driver = webdriver.Chrome(PATH)
driver = webdriver.Chrome(PATH) 
url = 'https://twitchtracker.com/ibai/streams'
driver.get(url)
columns = ['fecha', 'duracion', 'avg_ccv', 'max_ccv', 'seguidores', 'titulo']
data = []

max_pages = 15

# Recorrer las páginas
current_page = 1
while current_page <= max_pages:
    try:
        # Recopilar datos de la tabla
        nombre = driver.find_element(By.ID, 'streams')
        filas = nombre.find_elements(By.TAG_NAME, "tr")

        for fila in filas:
            # Encontrar las columnas dentro de la fila
            columnas = fila.find_elements(By.TAG_NAME, "td")

            # Extraer datos de las columnas específicas
            if len(columnas) > 0:
                fecha_str = columnas[0].text
                duracion = columnas[1].text.lower()
                avg_ccv = float(columnas[2].text.replace(",", ""))
                max_ccv = float(columnas[3].text.replace(",", ""))
                seguidores = float(columnas[4].text.replace(",", ""))
                titulo = columnas[6].text.lower()
                fecha = datetime.strptime(fecha_str, "%d/%b/%Y %H:%M")
                titulo = titulo.replace(" ", "_")


                # Agregar los datos a la lista
                data.append([fecha, duracion, avg_ccv, max_ccv, seguidores, titulo])

        # Pasar a la siguiente página
        next_page_button = driver.find_element(By.LINK_TEXT, str(current_page + 1))
        next_page_button.click()

        # Incrementar el contador de página
        current_page += 1

    except NoSuchElementException:
        break

# Crear DataFrame a partir de los datos obtenidos
df = pd.DataFrame(data, columns=columns)

# Imprimir el DataFrame con todos los datos recopilados
print(df.head())
    
# Cerrar el navegador
driver.quit()
df.info(memory_usage='deep')
df.to_csv('scrap_ibai_csv.csv', index=False)